In [2]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [10]:
df_train = pd.read_parquet("yellow_tripdata_2023-01.parquet")
initial_num_of_rows = df_train.shape[0]

In [11]:
df_train['duration'] = df_train.tpep_dropoff_datetime - df_train.tpep_pickup_datetime
df_train.duration = df_train.duration.apply(lambda td: td.total_seconds() / 60)

In [12]:
df_train["duration"].std()

42.59435124195458

In [13]:
df_train = df_train[(df_train.duration >= 1) & (df_train.duration <= 60)]

In [15]:
(df_train.shape[0]/initial_num_of_rows)*100

98.1220282212598

In [16]:
categorical = ('PULocationID', 'DOLocationID')
train_dicts = df_train.loc[:, categorical].astype(str).to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
del train_dicts

In [17]:
X_train.shape

(3009173, 515)

In [18]:
target = 'duration'
y_train = df_train[target].values
del df_train

In [19]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
mean_squared_error(y_pred, y_train, squared=False)

/home/aditya/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.649262236295703

In [20]:
df_val = pd.read_parquet('yellow_tripdata_2023-02.parquet')

In [21]:
df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

In [22]:
dv = DictVectorizer()
categorical = ('PULocationID', 'DOLocationID')
val_dicts = df_val.loc[:, categorical].astype('string').to_dict(orient='records')
X_val = dv.fit_transform(val_dicts)
del val_dicts

In [23]:
y_val = df_val['duration'].values
del df_val

In [24]:
X_val.shape

(2855951, 514)

In [25]:
y_pred = lr.predict(X_val)
mean_squared_error(y_pred, y_val, squared=False)

ValueError: X has 514 features, but LinearRegression is expecting 515 features as input.